***
## Кнопки для бота

В интерфейсе бота можно создавать кнопки с подписями. Нажатие на такую кнопку отправляет предустановленное разработчиком сообщение. Можно заготовить кнопки, с помощью которых пользователь в одно нажатие отправит любую команду или предустановленное текстовое сообщение. 

Можно создать кнопку, по нажатию которой будет отправлена команда /start; для получения нового котика можно создать кнопку, которая отправит команду /newcat.

***
## Создаём кнопки

Для управления кнопками в библиотеке pyTelegramBotAPI есть классы `types.ReplyKeyboardMarkup` и `types.KeyboardButton`.

Экземпляр класса `types.ReplyKeyboardMarkup` — это объект клавиатуры, он управляет кнопками. К объекту клавиатуры можно добавить сколько угодно кнопок — экземпляров класса `types.KeyboardButton`.


In [ ]:
# Создаём объект клавиатуры:
keyboard = types.ReplyKeyboardMarkup()
# Клавиатуру создали, но кнопок на ней пока нет.

# Создаём объект кнопки:
button_newcat = types.KeyboardButton('/newcat')
# Добавляем объект кнопки на клавиатуру:
keyboard.add(button_newcat) 

При создании объекта кнопки в конструктор класса передаётся строка, которая будет отправлена в сообщении при нажатии на эту кнопку. Эта же строка будет отображена на кнопке в качестве подписи (поставить другую подпись на кнопке невозможно).

Порядок создания кнопок на основе классов `types.ReplyKeyboardMarkup` и `types.KeyboardButton`:

1. Создать клавиатуру (экземпляр `types.ReplyKeyboardMarkup`).

2. Создать кнопки (экземпляры класса `types.KeyboardButton`).

3. Добавить кнопки на клавиатуру (например, методом `types.ReplyKeyboardMarkup.add()`).

4. Отправить клавиатуру пользователю — передать объект клавиатуры в параметр `reply_markup` метода `bot.send_message()`.

Объект клавиатуры отправляется пользователю в сообщении бота — и после получения такого сообщения в клиенте пользователя появятся кнопки клавиатуры.

***
## Кнопки в KittyBot

Импортируем в код модуль `types`, содержащий классы `ReplyKeyboardMarkup` и `KeyboardButton`, и добавим в интерфейс кнопку для команды */newcat*. Клавиатуру с этой кнопкой отправим вместе с ответом на команду */start*.


In [ ]:
# kittybot/kittybot.py

import requests
from telebot import TeleBot, types

bot = TeleBot(token='<token>')
URL = 'https://api.thecatapi.com/v1/images/search'


# Код запроса к thecatapi.com и обработку ответа обернём в функцию:
def get_new_image():
    response = requests.get(URL).json()
    random_cat = response[0].get('url')
    return random_cat


@bot.message_handler(commands=['newcat'])
def new_cat(message):
    chat = message.chat
    bot.send_photo(chat.id, get_new_image())


@bot.message_handler(commands=['start'])
def wake_up(message):
    chat = message.chat
    name = message.chat.first_name
    # Создаём объект клавиатуры:
    keyboard = types.ReplyKeyboardMarkup()
    # Создаём объект кнопки:
    button_newcat = types.KeyboardButton('/newcat')
    # Добавляем объект кнопки на клавиатуру:
    keyboard.add(button_newcat)

    bot.send_message(
        chat_id=chat.id,
        text=f'Привет, {name}. Посмотри, какого котика я тебе нашёл',
        # Отправляем клавиатуру в сообщении бота: передаём объект клавиатуры
        # в параметр reply_markup объекта send_message.
        # Telegram-клиент "запомнит" клавиатуру и будет отображать её в интерфейсе бота.
        reply_markup=keyboard,
    )

    bot.send_photo(chat.id, get_new_image())


@bot.message_handler(content_types=['text'])
def say_hi(message):
    chat = message.chat
    chat_id = chat.id
    bot.send_message(chat_id=chat_id, text='Привет, я KittyBot!')

bot.polling()

Кнопка с надписью /newcat передана и отображается, это успех! Однако она занимает на экране слишком много места: клиент Telegram автоматически растянул её. Высоту кнопки можно уменьшить: для этого в объекте клавиатуры нужно указать параметр `resize_keyboard=True`:


In [ ]:
keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True) 

С установленным параметром `resize_keyboard` высота кнопки будет зависеть от объёма текста, который на ней размещён:

Обработка сообщений, отправленных нажатием на кнопку, ничем не отличается от обработки любых других сообщений — ведь и сами сообщения ничем не различаются.

***
## Размещение кнопок в интерфейсе

На виртуальной клавиатуре бота может быть несколько кнопок, их, как правило, размещают в несколько строк. Библиотека pyTelegramBotAPI позволяет описать размещение кнопок на экране, и для этого есть два способа:

1. Метод `keyboard.add()` добавляет кнопки на клавиатуру. По умолчанию все кнопки выводятся в одну строку (в один ряд).

    Можно указать параметр `row_width=N`, где `N` — количество кнопок в строке, и тогда кнопки будут размещены в несколько рядов, по `N` кнопок в ряд.

    Например, `keyboard.add(row_width=3)` отрисует клавиатуру, где в строку помещается не более трёх кнопок. Чтобы расположить четыре кнопки на клавиатуре и установить правило «не более трёх кнопок на строке», добавление кнопок на клавиатуру надо описать так: `keyboard.add(button_1, button_2, button_3, button_4, row_width=3)`. Три кнопки окажутся в первой строке, и одна — во второй.

2. Метод `keyboard.row()` позволяет явным образом распределить кнопки по строкам: объявить строки и наполнить их кнопками.

Для примера опишем клавиатуру бота c помощью метода `.row()`. В него передаются кнопки, объекты класса `types.KeyboardButton`. Кнопки расположим в две строки: две в первой строке и одну — во второй.


In [ ]:
...

@bot.message_handler(commands=['start'])
def wake_up(message):
    chat = message.chat
    name = chat.first_name
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    keyboard.row(  # Первая строка кнопок.
        types.KeyboardButton('Который час?'),  # Создаём первую кнопку в строке.
        types.KeyboardButton('Определи мой ip'),  # Создаём вторую кнопку в строке.
    )
    keyboard.row(  # Вторая строка кнопок.
        types.KeyboardButton('/random_digit'),  # Создаём кнопку в строке.
    )
    
    bot.send_message(
        chat_id=chat.id,
        text=f'Спасибо, что вы включили меня, {name}!',
        reply_markup=keyboard,  # Отправляем пользователю текстовый ответ и клавиатуру.
    )
... 

![alt text](https://pictures.s3.yandex.net/resources/image_1709111174.png)

***
## Замена и удаление клавиатуры

Можно создать новую клавиатуру и отправить её — тогда новая заменит существующую. Можно удалить клавиатуру из интерфейса пользователя: для этого нужно отправить объект класса `types.ReplyKeyboardRemove` — существующий объект клавиатуры будет удалён, и кнопки пропадут.


In [ ]:
...
    bot.send_message(
        chat_id=chat.id,
        text='Удаляю клавиатуру!',
        # Удаляем клавиатуру из интерфейса пользователя:
        reply_markup=types.ReplyKeyboardRemove(),
    )
... 

>А вот и [шпаргалка по Telegram-ботам](https://code.s3.yandex.net/Python-dev/cheatsheets/040-api-telegram-boty-shpora/040-api-telegram-boty-shpora.html). Сохраняйте её в закладки. Вам предстоит написать ещё как минимум одного бота — в качестве финального проекта.